# 场景

1. 加入用一个多进程来更新一个数组的值
2. 对于同一位，存在A读取了数据值，然后B写入后，A等待3秒，对同一个地址进行写入，造成数据错误
3. 所以，需要一个锁，来对列表具体的每一个位置进行锁定，即一旦有进程读了该位置，那么其他进程只能等待
4. 多进程返回值

## 初始代码

1. 为什么列表的数据值没有改变

In [14]:
%%writefile lock_demo.py

import time
import os
from multiprocessing import Pool, cpu_count

def modify(lst, pos):
    print("I am here %d" %os.getpid())
    try:
        v = lst[pos]
        time.sleep(1)
        lst[pos] = v+1
    except Exception as ex:
        print(ex)
        traceback.print_exc()
    print("I am Finished %d" %os.getpid())
    
def process_modify(lst, pos_lst):
    print("Start %d" %os.getpid())
    try:
        for pos in pos_lst:
            modify(lst, pos)
    except Exception as ex:
        print(ex)
        traceback.print_exc()
    
    
if __name__ == "__main__":
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"
    pool = Pool(processes=cpu_count())
    
    lst = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    try:
        for i in range(6):
            print(i)
            pos_lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
            pool.apply_async(process_modify, args=(lst, pos_lst))
    except Exception as ex:
        print(ex)
        traceback.print_exc()
        
    pool.close()
    pool.join()
    
    print("Multiple Process is Finished!!!")
    
    print(lst)

Overwriting lock_demo.py


## 未加锁

1. [不同进程间，要想共享数据，必须使用Manager模块来实现](https://zhuanlan.zhihu.com/p/166091204)
2. [多进程共享，自定义数据类型](https://www.cnblogs.com/yangfantuan/p/12535436.html)


已经实现了list的更改，但是预期输出不对，也就是资源没有锁定，倒是重复读入数据和写入

In [2]:
# %run lock_demo.py

In [25]:
%%writefile lock_demo.py

import time
import os
import random
from multiprocessing import Pool, cpu_count, Manager

class Myclass:
    def __init__(self, data=None):
        self.data = data
        
    def get(self):
        return self.data
    
    def set(self, data):
        self.data = data

def modify(lst, pos):
    print("I am here %d" %os.getpid())
    try:
        v = lst[pos]
        time.sleep(1)
        lst[pos] = v+1
    except Exception as ex:
        print(ex)
        traceback.print_exc()
    print("I am Finished %d" %os.getpid())
    
def process_modify(lst, pos_lst):
    print("Start %d" %os.getpid())
    try:
        for pos in pos_lst:
            modify(lst, pos)
    except Exception as ex:
        print(ex)
        traceback.print_exc()
    
    
if __name__ == "__main__":
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"
    pool = Pool(processes=cpu_count())
    
    lst = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    #update
    manager = Manager()
    manager_list = manager.list(lst)
    
    try:
        for i in range(6):
            print(i)
            time.sleep(random.random())
            pos_lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
            pool.apply_async(process_modify, args=(manager_list, pos_lst))
    except Exception as ex:
        print(ex)
        traceback.print_exc()
        
    pool.close()
    pool.join()
    
    print("Multiple Process is Finished!!!")
    
    print(lst)
    print(manager_list)

Overwriting lock_demo.py


多进程，共享自定义数据类型

In [ ]:
# %%writefile demo.py
import threading
import multiprocessing
from multiprocessing.managers import BaseManager

class Data:
    def __init__(self,data=None):
        self.data = data

    def get(self):
        return self.data

    def set(self,data):
        self.data = data
        
BaseManager.register("mydata",Data)

def test(name,data,lock):
    lock.acquire()
    print("in thread {} name is {}".format(threading.current_thread(),name))
    print("data is {} id(data) is {}".format(data,id(data)))
    data.set(data.get()+1)
    lock.release()



def getManager():
    m = BaseManager()
    m.start()
    return m


if __name__ == '__main__':
    manager = getManager()
    d = manager.mydata(0)
    thlist = []
    name = "yang"
    lock = multiprocessing.Lock()
    for i in range(5):
        th = multiprocessing.Process(target=test,args=(name,d,lock))
        th.start()
        thlist.append(th)
    for i in thlist:
        i.join()
    print(d.get())

In [2]:
%run demo.py # 在这里执行，不会打印执行结果，需要在terminal中打印

5


## 加锁

[Python中各种锁](https://www.jianshu.com/p/083bbe5f5534)

In [28]:
%%writefile lock_demo.py

import time
import os
import random
from multiprocessing import Pool, cpu_count, Manager, Lock

def modify(lst, pos):
    print("I am here %d" %os.getpid())
    try:
        v = lst[pos]
        time.sleep(1)
        lst[pos] = v+1
    except Exception as ex:
        print(ex)
        traceback.print_exc()
    
        
    print("I am Finished %d" %os.getpid())
    
def process_modify(lst, pos_lst, lock):
    print("Start %d" %os.getpid())
    lock.acquire()

    try:
        for pos in pos_lst:
            modify(lst, pos)
    except Exception as ex:
        print(ex)
        traceback.print_exc()
    finally:
        lock.release()
    
    
if __name__ == "__main__":
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"
    pool = Pool(processes=cpu_count())
    
    lst = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    #update
    manager = Manager()
    manager_list = manager.list(lst)
    
    #update 2
    lock = Lock()
    
    try:
        for i in range(6):
            print(i)
            time.sleep(random.random())
            pos_lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
            pool.apply_async(process_modify, args=(manager_list, pos_lst, lock))
    except Exception as ex:
        print(ex)
        traceback.print_exc()
        
    pool.close()
    pool.join()
    
    print("Multiple Process is Finished!!!")
    
    print(lst)
    print(manager_list)

Overwriting lock_demo.py


## 多进程返回值

In [ ]:
def worker(procnum, return_dict):
    '''worker function'''
    print str(procnum) + ' represent!'
    return_dict[procnum] = procnum

if __name__ == '__main__':
    manager = Manager()
    return_dict = manager.dict()
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,return_dict))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    # 最后的结果是多个进程返回值的集合
    print return_dict.values()

In [21]:
import random